In [1]:
import re
import ast

# 读取文件
# with open('en_arg_seed.txt', 'r') as file:
# with open('zh_arg_seed.txt', 'r') as file:
with open('en_slm_seed.txt', 'r') as file:
# with open('zh_slm_seed.txt', 'r') as file:
    file_contents = file.read()

# 使用正则表达式找到所有符合格式的行
pattern = re.compile(r"the_test_metric: ({.*?})")
matches = pattern.findall(file_contents)

# 使用 ast.literal_eval 安全地将匹配的字符串转换为字典
dict_list = [ast.literal_eval(match) for match in matches]

# dict_list 就是包含所有指标字典的列表


In [2]:
from pprint import pprint
from statistics import mean, stdev
def get_dict_mean_stdev(dict_list, digits=None):

    # 对某个key做操作
    def recurse_dicts(dict_list, current_key):
        # 提取这个键对应的所有值（可能是字典或数值）
        value_list = [
            current_dict[current_key]
            for current_dict in dict_list
            if current_key in current_dict
        ]

        # 如果值是字典，则递归处理
        if isinstance(value_list[0], dict):
            return get_dict_mean_stdev(value_list, digits)

        # 计算均值和标准差
        calculated_mean = mean(value_list)
        calculated_stdev = stdev(value_list) if len(value_list) > 1 else 0

        # 如果digits不为None，则四舍五入
        if digits is not None:
            calculated_mean = round(calculated_mean, digits)
            calculated_stdev = round(calculated_stdev, digits)

        return {"mean": calculated_mean, "stdev": calculated_stdev}

    # 初始化结果字典
    result = {}

    # 获取所有字典的键的并集
    keys = set().union(*[d.keys() for d in dict_list])

    # 对每个键进行操作
    for key in keys:
        result[key] = recurse_dicts(dict_list, key)

    return result

pprint(get_dict_mean_stdev(dict_list,3))

{'acc': {'mean': 0.867, 'stdev': 0.004},
 'auc': {'mean': 0.85, 'stdev': 0.005},
 'f1_fake': {'mean': 0.625, 'stdev': 0.01},
 'f1_real': {'mean': 0.919, 'stdev': 0.003},
 'metric': {'mean': 0.772, 'stdev': 0.006},
 'precision': {'mean': 0.784, 'stdev': 0.008},
 'precision_fake': {'mean': 0.659, 'stdev': 0.017},
 'precision_real': {'mean': 0.91, 'stdev': 0.004},
 'recall': {'mean': 0.763, 'stdev': 0.008},
 'recall_fake': {'mean': 0.596, 'stdev': 0.019},
 'recall_real': {'mean': 0.929, 'stdev': 0.007},
 'spauc': {'mean': 0.739, 'stdev': 0.012}}
